In [3]:
import json
import torch
import pickle
import numpy as np
import torch.nn as nn
from scipy import stats
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from nn_utils import Attention, Discriminator, EmotionRegression, FeatureExtaction

In [4]:
batch_size = 64
lr_attn = 1e-4
lr_feature = 8e-5
lr_regressor = 4e-5
lr_discriminator = 4e-5
epochs = 10
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [5]:
dict = {}
with open("dataset.pkl", "rb") as fin :
    dict = pickle.load(fin)

vocab = dict['vocab'].to(device)
train_dataloader = DataLoader(
    dict['train_dataset'],
    shuffle=True,
    batch_size=batch_size
    )
val_dataloader = DataLoader(
    dict['dev_dataset'],
    shuffle=False,
    batch_size=batch_size
    )

In [6]:
class AAN(nn.Module) :
    def __init__(self, embed_size=300, hidden_size=150) :
        #['V', 'A', 'D', 'S']
        super(AAN, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.feature_size = self.hidden_size*4

        self.attention_1 = Attention(self.embed_size)
        self.attention_2 = Attention(self.embed_size)
        self.attention_s = Attention(self.embed_size)

        self.features = FeatureExtaction(self.embed_size, self.hidden_size)
        
        self.regression_1 = EmotionRegression(self.feature_size * 2)
        self.regression_2 = EmotionRegression(self.feature_size * 2)

        self.discriminator = Discriminator(self.feature_size)
    
    def forward(self, vocab, sentences, source_lengths) :
        sentences = vocab(sentences)
        sentences_1 = self.attention_1(sentences, source_lengths)
        sentences_2 = self.attention_2(sentences, source_lengths)
        sentences_s = self.attention_s(sentences, source_lengths)

        features_1 = self.features(sentences_1, source_lengths)
        features_2 = self.features(sentences_2, source_lengths)
        features_s = self.features(sentences_s, source_lengths)

        value_1 = self.regression_1(torch.cat((features_1, features_s), dim=1))
        value_2 = self.regression_2(torch.cat((features_2, features_s), dim=1))
        
        p1, p2 = self.discriminator(features_1), self.discriminator(features_2)

        return value_1, value_2, p1, p2


In [7]:
class Train() :
    def __init__(self, type=[0,1]) :
        super(Train, self).__init__()
        self.type = type
        self.model = AAN().to(device)
        # self.init_weights()
        self.mse = nn.MSELoss()
        self.attention_optim = optim.Adam(
            list(self.model.attention_1.parameters())+
            list(self.model.attention_2.parameters())+
            list(self.model.attention_s.parameters()), lr=lr_attn)
        self.feature_optim = optim.Adam(self.model.features.parameters(), lr=lr_feature)
        self.regressor_optim = optim.RMSprop(
            list(self.model.regression_1.parameters())+
            list(self.model.regression_1.parameters()), lr=lr_regressor)
        self.discriminator_optim = optim.RMSprop(self.model.discriminator.parameters(), lr=lr_discriminator)
        self.training_stats = []
    
    def init_weights(self) :
        for param in self.model.parameters() :
            temp = 0
            try :
                temp = np.sqrt(6.0/(param.shape[0]+param.shape[1]+1e-8))
            except: 
                pass
            param.data.uniform_(-temp, temp)
    
    def run_model(self, batch) :
        sentences = batch[0].to(device)
        source_lengths = batch[1].to(device)
        target = batch[2].to(device)
        value_1, value_2, p1, p2 = self.model(vocab, sentences, source_lengths)
        output = torch.cat((value_1, value_2), dim=1)
        target = target[:,self.type].float()
        return output, p1, p2, target
    
    def get_r(self, output, target) :
        temp = [(stats.pearsonr(output[:,i].cpu().detach(), target[:,i].cpu().detach())) for i in range(2)]
        return temp
    
    def train(self, epochs=epochs) :
        mse = nn.MSELoss()
        for epoch_i in range(epochs) :
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            self.model.train()
            train_r = list()
            train_loss = list()
            for i, batch in enumerate(train_dataloader) :

                self.attention_optim.zero_grad()
                self.feature_optim.zero_grad()
                self.regressor_optim.zero_grad()

                output, p1, p2, target = self.run_model(batch)
                reg_loss = mse(output, target)
                train_loss.append(reg_loss.item())
                reg_loss.backward(retain_graph=True)
                self.attention_optim.step()
                self.feature_optim.step()
                self.regressor_optim.step()

                self.discriminator_optim.zero_grad()

                wloss = (p2-p1).mean()
                wloss.backward(retain_graph=True)
                self.discriminator_optim.step()

                self.attention_optim.zero_grad()

                adversarial_loss = (p1 - p2).mean()
                adversarial_loss.backward()
                self.attention_optim.step()
                # train_r = train_r.append(self.get_r(output, target))
                if i%10 == 0 :
                    print("Batch: {} train Loss: {}".format(i, reg_loss))

            val_loss = []
            val_r = []
            for i, batch in enumerate(val_dataloader) :
                with torch.no_grad() :
                    output, p1, p2, target = self.run_model(batch)

                    reg_loss = mse(output, target)
                    val_loss.append(reg_loss.item())
                    # val_r = val_r.append(self.get_r(output, target))
                    if i%10 == 0 :
                        print("Batch: {} val Loss: {}".format(i, reg_loss))

            self.training_stats.append({
                'training loss' : sum(train_loss)/len(train_loss),
                'validation loss' : sum(val_loss)/len(val_loss),
                # 'train r' : torch.tensor(train_r).mean(dim=0),
                # 'val r' : torch.tensor(val_r).mean(dim=0),
            })
            print(json.dumps(self.training_stats[-1], indent=4))
            
        def save_model(self) :
            #TODO: Add function to save model
            pass
        
        def plot(self, r_values) :
            #TODO: Plot r values
            pass

In [8]:
train = Train()
train.train(10)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.0
0.0
0.0
0.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.0
0.0
0.0
0.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.0
0.0
0.0
0.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
